In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as sst
import os

from sklearn.linear_model import LinearRegression

%matplotlib inline

In [ ]:
parent_dir = os.path.split(os.getcwd())[0]

In [ ]:
zones = gpd.read_file(parent_dir + '\\Data\\New\\lms_zone_du_new.shp') # LMS Zone data

In [ ]:
ovin = pd.read_csv(parent_dir + '\\Data\\New\\Ovin_final.csv', index_col=0)

#### OD matrices

In [ ]:
car_driver = pd.read_csv(parent_dir + '\\Data\\New\\LMS_results\\lms-car-driver', index_col=0)
car_pass = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\PAS_etmaal.dat', delimiter='\t')
train = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\TREIN_etmaal.dat', delimiter='\t')
bus = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\BUS_etmaal.dat', delimiter='\t')
hov = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\HOV_etmaal.dat', delimiter='\t')
bike = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\FIETS_etmaal.dat', delimiter='\t')
ebike = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\EBIKE_etmaal.dat', delimiter='\t')
walk = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\LOOP_etmaal.dat', delimiter='\t')

In [ ]:
bike_final = bike.merge(ebike, on=['Orig', 'Dest'], how='outer')
bike_final['TRP'] = bike_final['TRP_x'] + bike_final['TRP_y']
bike_final = bike_final[['Orig', 'Dest', 'TRP']]

### Travel times

In [ ]:
car_time = pd.read_csv(parent_dir + "\\Data\\LMS\\LOS_correct\\reistijd_auto_etmaal.csv", delimiter=';',
                        header=None, names=['Orig', 'Dest', 'Time'], decimal=',')

In [ ]:
ov_time = pd.read_csv(parent_dir + "\\Data\\LMS\\LOS_correct\\reistijd_ov_etmaal.csv", delimiter=';',
                        header=None, names=['Orig', 'Dest', 'Time'], decimal=',')

In [ ]:
bike_time = pd.read_csv(parent_dir + "\\Data\\LMS\\LOS_correct\\reistijd_fiets_etmaal.csv", delimiter=';',
                        header=None, names=['Orig', 'Dest', 'Time'], decimal=',')

#### Distances

In [ ]:
car_dist = pd.read_csv(parent_dir + '\\Data\\LMS\LOS_correct\\afstand_auto_freeflow.csv', delimiter=';', header=None, 
                       names=['Orig', 'Dest', 'Dist'], decimal=',')

car_dist['Dist'] = car_dist['Dist'] / 1000

In [ ]:
btm_dist = pd.read_csv(parent_dir + "\\Data\\LMS\\LOS_correct\\LOS9_118_LMS.ASC", delimiter=r"\s+", header=None,
                       names=['Orig', 'Dest', 'Bus_time_il', 'bus_time_jl', 'bus_time_b', 'bus_time_tm', 'bus_wait_i', 'bus_time_o', 'bus_dist_ov', 'bus_dist_nt', 'bus_o', 'bus_cost',
                              'tm_time_il', 'tm_time_jl', 'tm_time_b', 'tm_time_tm', 'tm_iwait', 'tm_time_o', 'tm_dist_ov', 'tm_dist_nt', 'tm_over', 'tm_cost'])


In [ ]:
btm_dist = btm_dist[['Orig', 'Dest', 'bus_dist_ov', 'bus_dist_nt', 'tm_dist_ov', 'tm_dist_nt']]

In [ ]:
bike_dist = pd.read_csv(parent_dir + "\\Data\\LMS\\LOS_correct\\FIETS_LOOP\\LOS6__18LMS_fiets_km.txt", delimiter=r"\s+", header=None,
                        names=['Orig', 'Dest', 'Dist'])

Get intrazonal distance

In [ ]:
inter_dist = bike_dist.groupby('Orig').min()

In [ ]:
idx = bike_dist.groupby('Orig').idxmin().Dist
bike_dist.loc[idx]

In [ ]:
intra_dist = 0.5 * np.sqrt(zones.OPP / 100 /  np.pi)
intra_dist

In [ ]:
F = np.array(intra_dist) / np.array(inter_dist.Dist)
F

In [ ]:
intra_dist_arr = np.vstack([np.arange(1, 1407), np.arange(1, 1407), intra_dist]).T

In [ ]:
intra_dist_pd = pd.DataFrame(intra_dist_arr, columns=['Orig', 'Dest', 'Dist'])

In [ ]:
bike_dist = pd.concat([bike_dist, intra_dist_pd])

Calculate intrazonal travel time bike

In [ ]:
intra_time = bike_time.loc[idx].Time * F
intra_time_arr = np.vstack([np.arange(1, 1407), np.arange(1, 1407), intra_time]).T
intra_time_pd = pd.DataFrame(intra_time_arr, columns=['Orig', 'Dest', 'Time'])
bike_time = pd.concat([bike_time, intra_time_pd])

Remove trips to and from other countries

In [ ]:
mode_list = [car_driver, car_pass, train, bus, hov, bike_final, walk]

for i in range(len(mode_list)):
    mode_list[i] = mode_list[i][(mode_list[i].Orig.isin(np.arange(1, 1407))) & (mode_list[i].Dest.isin(np.arange(1, 1407)))]

In [ ]:
tb_data = zones[['ZONE_ID']] # Create df to store results

In [ ]:
tb_data = tb_data.set_index('ZONE_ID')

### Get distance for each zone

car

In [ ]:
mode_list[0] = mode_list[0].merge(car_dist, on=['Orig', 'Dest'], how='left')
mode_list[0]['tot_dist'] = mode_list[0].tot * mode_list[0].Dist
grouped = mode_list[0].groupby('Orig').sum()
ave_dist = grouped.tot_dist / grouped.tot
tb_data['Car_driver_dist_o'] = ave_dist

Car passenger

In [ ]:
mode_list[1] = mode_list[1].merge(car_dist, on=['Orig', 'Dest'], how='left')
mode_list[1]['tot_dist'] = mode_list[1].TRP * mode_list[1].Dist
grouped = mode_list[1].groupby('Orig').sum()
ave_dist = grouped.tot_dist / grouped.TRP
tb_data['Car_pass_dist_o'] = ave_dist


bus

In [ ]:
btm_dist

In [ ]:
btm = mode_list[3].merge(mode_list[4], on=['Orig', 'Dest'], how='outer')
btm['TRP'] = btm[['TRP_x', 'TRP_y']].sum(axis=1)
btm

In [ ]:
btm = btm.merge(btm_dist, on=['Orig', 'Dest'], how='left')

In [ ]:
btm.dropna()

In [ ]:
btm['dist_bus'] = btm[['bus_dist_ov', 'bus_dist_nt']].sum(axis=1)
btm['dist_tm'] = btm[['tm_dist_ov', 'tm_dist_nt']].sum(axis=1)


In [ ]:
## btm added to df later

Train. To simplify, first determine travel time based on distance using linear regression on OViN data. Because travel times OV are known. If results seem interesting, do a more in depth analysis.

In [ ]:
tr = ovin[ovin.KHvm.isin([3])][['AfstV', 'VertZone', 'AankZone', 'KHvm', 'Reisduur']]

In [ ]:
tr

In [ ]:
tr_afst = np.vstack([np.ones(len(tr)), tr.AfstV / 10]).T

In [ ]:
regr = LinearRegression()
regr.fit(tr_afst, tr.Reisduur / 60)

In [ ]:
x = np.arange(0, 400, 100)

plt.scatter(tr.AfstV / 10, tr.Reisduur / 60, alpha=0.1)
plt.plot(x, regr.intercept_ + x * regr.coef_[1], color='red', linestyle='--')
plt.xlabel('Distance [km]')
plt.ylabel('Travel time [h]')

In [ ]:
ov_time['train_dist'] = (ov_time.Time / 60 - regr.intercept_) / regr.coef_[1]

In [ ]:
mode_list[2] = mode_list[2].merge(ov_time, on=['Orig', 'Dest'], how='left')
mode_list[2]['tot'] = mode_list[2].TRP * mode_list[2].train_dist
grouped = mode_list[2].groupby('Orig').sum()
ave_dist = grouped.tot / grouped.TRP
tb_data['Train_dist_o'] = ave_dist


btm

In [ ]:
btm['tot_bus'] = btm.TRP_x * btm.dist_bus
btm['tot_tm'] = btm.TRP_y * btm.dist_tm
btm['tot'] = btm[['tot_bus', 'tot_tm']].sum(axis=1)
grouped = btm.groupby('Orig').sum()
ave_dist = grouped.tot / grouped.TRP
tb_data['BTM_dist_o'] = ave_dist

In [ ]:
tb_data

Biking

In [ ]:
mode_list[5] = mode_list[5].merge(bike_dist, on=['Orig', 'Dest'], how='left')

In [ ]:
mode_list[5]['tot'] = mode_list[5].TRP * mode_list[5].Dist
grouped = mode_list[5].groupby('Orig').sum()
ave_dist = grouped.tot / grouped.TRP
tb_data['Bike_dist_o'] = ave_dist


Walking

In [ ]:
mode_list[6] = mode_list[6].merge(bike_dist, on=['Orig', 'Dest'], how='left')

In [ ]:
mode_list[6]['tot'] = mode_list[6].TRP * mode_list[6].Dist
grouped = mode_list[6].groupby('Orig').sum()
ave_dist = grouped.tot / grouped.TRP
tb_data['Walk_dist_o'] = ave_dist

In [ ]:
tb_data

## Travel time

Car driver

In [ ]:
mode_list[0] = mode_list[0].merge(car_time, on=['Orig', 'Dest'], how='left')


In [ ]:
mode_list[0]['tot_time'] = mode_list[0].tot * mode_list[0].Time
grouped = mode_list[0].groupby('Orig').sum()
ave_dist = grouped.tot_time / grouped.tot
tb_data['Car_driver_time_o'] = ave_dist


Car passenger

In [ ]:
mode_list[1] = mode_list[1].merge(car_time, on=['Orig', 'Dest'], how='left')

In [ ]:
mode_list[1]['tot_time'] = mode_list[1].TRP * mode_list[1].Time
grouped = mode_list[1].groupby('Orig').sum()
ave_dist = grouped.tot_time / grouped.TRP
tb_data['Car_pass_time_o'] = ave_dist


Train

In [ ]:
mode_list[2]['tot_time'] = mode_list[2].TRP * mode_list[2].Time
grouped = mode_list[2].groupby('Orig').sum()
ave_dist = grouped.tot_time / grouped.TRP
tb_data['Train_time_o'] = ave_dist

btm

In [ ]:
btm = btm.merge(ov_time, on=['Orig', 'Dest'], how='left')

In [ ]:
btm['tot_time'] = btm.TRP * btm.Time
grouped = btm.groupby('Orig').sum()
ave_dist = grouped.tot_time / grouped.TRP
tb_data['BTM_time_o'] = ave_dist

Bike

In [ ]:
mode_list[5] = mode_list[5].merge(bike_time, on=['Orig', 'Dest'], how='left')

In [ ]:
mode_list[5]['tot_time'] = mode_list[5].TRP * mode_list[5].Time
grouped = mode_list[5].groupby('Orig').sum()
ave_dist = grouped.tot_time / grouped.TRP
tb_data['Bike_time_o'] = ave_dist

Walk

In [ ]:
tb_data['Walk_time_o'] = tb_data.Walk_dist_o / 5 * 60

In [ ]:
tb_data = np.round(tb_data, 2)

## save data

In [ ]:
tb_data.to_csv(parent_dir + '\\Data\\New\\lms_zone_lms_tt_td.csv')